##**Maestría en Inteligencia Artificial Aplicada**
###**Curso: Inteligencia Artificial y Aprendizaje Automático**
####Tecnológico de Monterrey
####Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
####**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Josue Aguila Ramos
*   01796400


In [2]:
# Agrega aquí todas las librerías y paquetes adicionales que requieras.

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt  # Para visualizar datos y resultados
import seaborn as sns  # Gráficos más estéticos y análisis exploratorio
import scipy.sparse  # Para manejar eficientemente matrices dispersas
import warnings
warnings.filterwarnings("ignore")  # Para evitar advertencias molestas en Jupyter


### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data

### **Puedes también usar la siguiente liga:**

https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset

In [3]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data_rating = pd.read_csv("rating_final.csv", header='infer', sep=",")
data_geo = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data_rating.shape, data_geo.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El propósito de usar el argumento "latin-1" al cargar el archivo geoplaces2.csv es manejar correctamente la codificación de caracteres. En muchos casos, los archivos CSV contienen caracteres especiales (como tildes o eñes en español), y la codificación predeterminada en pandas es UTF-8. Sin embargo, si el archivo fue guardado con otra codificación, como ISO-8859-1 (también llamado latin-1), intentar leerlo sin especificar la codificación adecuada podría generar errores o caracteres mal interpretados.

++++++++ Termina la sección de agregar texto. +++++++++++

## **Ejercicio - 2**

#### **Cada restaurante debe tener un nombre diferente:**

In [4]:
# Veamos la lista de restaurantes.
# Todos ellos son restaurantes ubicados en alguna ciudad del país de México:

data_geo['name'].values

array(['Kiku Cuernavaca', 'puesto de tacos', 'El Rincón de San Francisco',
       'little pizza Emilio Portes Gil', 'carnitas_mata',
       'Restaurant los Compadres', 'Taqueria EL amigo ', 'shi ro ie',
       'Pollo_Frito_Buenos_Aires', 'la Estrella de Dimas',
       'Restaurante 75', 'Abondance Restaurante Bar',
       'El angel Restaurante', 'Restaurante Pueblo Bonito',
       'Mcdonalds Parque Tangamanga', 'Tortas y hamburguesas el gordo',
       'Sirlone', 'rockabilly ', 'Unicols Pizza', 'TACOS EL GUERO',
       'Restaurant El Muladar de Calzada', 'La Posada del Virrey',
       'Restaurant and Bar and Clothesline Carlos N Charlies', 'KFC',
       'Giovannis', 'Restaurant Oriental Express', 'Mariscos Tia Licha',
       'cafe ambar', 'Restaurante la Gran Via', 'don burguers',
       'Restaurante y Pescaderia Tampico', 'Rincon del Bife',
       'La Fontana Pizza Restaurante and Cafe',
       'Restaurante la Estrella de Dima', 'El Rincon de San Francisco',
       'Preambulo Wifi Zone 

*   ### **Todos los restaurantes que aparecen en esta lista son negocios diferentes, en el sentido de que físicamente están ubicados en diferentes ciudades de México, aunque algunos pertenezcan a una franquicia, como Vips.**

*   ### **Para los fines de este ejercicio, estaremos considerando a todos los restaurantes como diferentes. Sin embargo, en el caso de Vips, existen tres restaurantes en esta lista capturados con los nombres: VIPS, Vips y vips. Que por sus coordenadas de latitud y longitud sabemos que están ubicados en SLP, Cuernavaca y cdVictoria. Se podrían cambiar los nombres agregando la ciudad, pero como se capturaron los nombres de manera diferente con minúsculas y mayúsculas, los dejaremos por el momento de esta manera.**

*   ### **Por el momento no estaremos haciendo las recomendaciones por el lugar donde se encuentran, sino solamente por la evaluación del restaurante y otras características que no tienen que ver con la ubicación.**

*   ### **Existen otras franquicias capturadas también de manera diferente, salvo el caso de 'Gorditas Dona Tota', en el cual dos de estos restaurantes que están ubicados en Cd.Victoria fueron registrados exactamente con el mismo nombre. Para que nuestro sistema de recomendación pueda diferenciarlos, deberás cambiar el nombre de uno de ellos como se te indica a continuación.**  

**De los dos restaurantes registrados con el nombre de 'Gorditas Dona Tota', encuentra sus índices y cambia el que tiene el mayor índice en data_geo[name] al de  'Gorditas Dona Tota 2'.**

In [5]:
# Ejercicio-2

# ************* Incluye aquí tu código:*****************************

# Filtrar los índices donde el nombre es 'Gorditas Dona Tota'
indices_Dona_Tota = data_geo[data_geo['name'] == 'Gorditas Dona Tota'].index

# Verificar si hay más de un restaurante con ese nombre
if len(indices_Dona_Tota) > 1:
    # Obtener el índice más alto
    max_index = indices_Dona_Tota.max()

    # Cambiar el nombre en el índice más alto
    data_geo.loc[max_index, 'name'] = 'Gorditas Dona Tota 2'

# *********** Aquí termina la sección de agregar código *************

print('Verificando el cambio de nombre:\n', data_geo.loc[indices_Dona_Tota, 'name'])


Verificando el cambio de nombre:
 89       Gorditas Dona Tota
118    Gorditas Dona Tota 2
Name: name, dtype: object


## **Ejercicio - 3**

In [6]:
# Al momento tenemos los siguientes DataFrames. Del primer archivo
# tenemos la evaluación general, la de la comida y la del servicio:

data_rating.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In [7]:
# Y del segundo archivo obtenemos información diversa de cada restaurante:

data_geo.head(2).T

,0,1
placeID,134999,132825
latitude,18.915421,22.147392
longitude,-99.184871,-100.983092
the_geom_meter,0101000020957F000088568DE356715AC138C0A525FC46...,0101000020957F00001AD016568C4858C1243261274BA5...
name,Kiku Cuernavaca,puesto de tacos
address,Revolucion,esquina santos degollado y leon guzman
city,Cuernavaca,s.l.p.
state,Morelos,s.l.p.
country,Mexico,mexico
fax,?,?


### **De cada uno de estos archivos selecciona y combina las variables adecuadas para obtener un nuevo DataFrame cuyas columnas sean el ID de usuario (userID), el ID del restaurante (placeID), la calificación general (rating) y el nombre del restaurante (name). A este nuevo DataFrame llamarlo df_combinado.**

In [8]:
# Ejercicio 3:

# ************* Incluye aquí tu código:*****************************

# Cargar los archivos CSV
data_ratings = pd.read_csv("rating_final.csv")  # Contiene userID, placeID y rating
data_geo = pd.read_csv("geoplaces2.csv", encoding="latin-1")  # Contiene placeID y name

# Combinar ambos DataFrames utilizando placeID como clave
df_combinado = pd.merge(data_ratings, data_geo[['placeID', 'name']], on='placeID', how='left')

# *********** Aquí termina la sección de agregar código *************

# Despleguemos la dimensión y los primeros renglones de este DataFrame:
print(df_combinado.shape)
df_combinado.head()


(1161, 6)


,userID,placeID,rating,food_rating,service_rating,name
0,U1077,135085,2,2,2,Tortas Locas Hipocampo
1,U1077,135038,2,2,1,Restaurant la Chalita
2,U1077,132825,2,2,2,puesto de tacos
3,U1077,135060,1,2,2,Restaurante Marisco Sam
4,U1068,135104,1,1,2,vips


from matplotlib import pyplot as plt
_df_0['placeID'].plot(kind='hist', bins=20, title='placeID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['food_rating'].plot(kind='hist', bins=20, title='food_rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['name'].value_counts()
    for x_label, grp in _df_13.groupby('userID')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('userID')
_ = plt.ylabel('name')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['userID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='placeID', y='userID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='placeID', y='name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['userID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='rating', y='userID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='rating', y='name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## **Ejercicio - 4**

In [9]:
# Ejercicio 4:

#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación general (rating). La llamaremos "UtMx_rating".


# ************* Incluye aquí tu código:*****************************

# Crear la matriz de utilidad usando pivot_table
UtMx_rating = df_combinado.pivot_table(index='name', columns='userID', values='rating', aggfunc='mean')

# *********** Aquí termina la sección de agregar código *************

# Mostrar la dimensión y las primeras filas de la matriz de utilidad
print('Dimensión de la matriz de Utilidad:')
print('(restaurantes, usuarios) =', UtMx_rating.shape)
UtMx_rating.head()


Dimensión de la matriz de Utilidad:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### **Similaridades con la matriz obtenida de la Factorización SVD Truncada:**

*   ### **Aplicaremo la Factorización SVD con los 66 primeros valores singulares a la matriz de utilidad UtMx_rating.**

*   ### **El valor de 66 se seleccionó porque como verás, esta cantidad de valores singulares es suficiente para describir al menos el 95% de la varianza de la matriz de utilidad.**

*   ### **Recordemos que la factorización SVD de una matriz $A$ tiene la forma:** $A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

*   ### **Usaremos la función coseno como medida de similaridad**

*   ### **Buscaremos restaurantes similares al llamado  "tacos de barbacoa enfrente del Tec"**.

In [11]:
# Solución: Reemplazar valores NaN en la matriz de utilidad antes de aplicar SVD
UtMx_rating_filled = UtMx_rating.fillna(0)  # Opcionalmente, podríamos usar .fillna(UtMx_rating.mean(axis=1), axis=0)

num_componentes = min(66, UtMx_rating_filled.shape[1])  # Ajustamos al número máximo de columnas posibles

# Aplicamos SVD truncada
SVD_rating = TruncatedSVD(n_components=num_componentes)
mat_fact_lat_restaurantes = SVD_rating.fit_transform(UtMx_rating_filled)

# Imprimimos dimensiones
print('Dimensión Vectores latentes restaurantes:', mat_fact_lat_restaurantes.shape)
print('Dimensión Traspuesta Vectores latentes usuarios:', SVD_rating.components_.shape)

# Variabilidad acumulada de las componentes utilizadas
print('\nVariabilidad acumulada de las componentes utilizadas: %.3f' % SVD_rating.explained_variance_ratio_.sum())

# Matriz de similaridad
sim_matrix = cosine_similarity(mat_fact_lat_restaurantes)

# Restaurante de referencia
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"
nombres_rest = UtMx_rating_filled.index  # Lista de los nombres de los restaurantes
if restaurante_de_referencia in nombres_rest:
    idx_rest = list(nombres_rest).index(restaurante_de_referencia)  # Índice del restaurante de referencia
    sim_vector = sim_matrix[idx_rest]  # Vector de similaridad

    # Buscando las similaridades positivas
    mejores_sim_rate = [(sim_vector[i], nombres_rest[i]) for i in range(len(nombres_rest)) if sim_vector[i] > 0]

    print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))

    # Ordenar de mayor a menor relevancia
    mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x: x[0], reverse=True)

    print('\nMejores recomendaciones con base al restaurante: \"%s\":' % restaurante_de_referencia)
    mejores_sim_rate_ordenadas[1:11]  # Omitimos el primero porque es el mismo restaurante de referencia
else:
    print(f'El restaurante "{restaurante_de_referencia}" no se encuentra en la base de datos.')


Dimensión Vectores latentes restaurantes: (129, 66)
Dimensión Traspuesta Vectores latentes usuarios: (66, 138)

Variabilidad acumulada de las componentes utilizadas: 0.954

Total de similaridades positivas encontradas: 68

Mejores recomendaciones con base al restaurante: "tacos de barbacoa enfrente del Tec":


## **Ejercicio - 5**

### **Sistema Híbrido**

### **Utilicemos ahora un sistema híbrido, el cual consistirá en conjuntar la matriz de "factores_latentes_restaurantes" (que es un ndarray) obtenida previamente, con la matriz de factores adicionales (que también será un ndarray) formada por las columnas One-Hot-Encoder de los factores 'dress_code', 'accessibility', 'price' y 'franchise' del DataFrame data_geo.**

### **Por el momento no requerimos que la matriz híbrida sea un DataFrame, con que sea un arreglo de NumPy, ndarray, será suficiente.**

### **Los renglones de la matriz híbrida siguen representando a cada restaurante, pero ahora con la información de los vectores latentes de la factorización SVD, más los vectores OneHotEncoding de los nuevos factores que se incluyeron.**



In [17]:
# Ejercicio 5:

# Define el DataFrame "df_fact_adicionales" formada por los factores 'dress_code',
# 'accessibility', 'price' y 'franchise'.
# Después generar la matriz (ndarray), "mat_fact_adicionales_ohe", que se obtiene
# al tranformar las columnas (variables categóricas nominales) de "df_fact_adicionales"
# mediante OneHotEncoding.
# Finalmente conjuntar horizontalmente la matriz "mat_fact_lat_restaurantes"
# con "mat_fact_adicionales_ohe" para obtener la "matriz_hibrida" (ndarray).
# Toma en cuenta que "matriz_hibrida" estará formada por las columnas de "mat_fact_lat_restaurantes"
# seguida por las columnas de "mat_fact_adicionales_ohe".


# ************* Incluye aquí tu código:*****************************

# 1. Filtrar `data_geo` para que solo contenga los restaurantes presentes en `UtMx_rating`
data_geo_filtrado = data_geo[data_geo['name'].isin(UtMx_rating.index)].copy()

# 2. Eliminar duplicados para asegurar que cada restaurante aparece solo una vez
data_geo_filtrado = data_geo_filtrado.drop_duplicates(subset='name')

# 3. Asegurar que `data_geo_filtrado` tenga exactamente los mismos restaurantes en el mismo orden que `UtMx_rating`
data_geo_filtrado = data_geo_filtrado.set_index("name").loc[UtMx_rating.index].reset_index()

# 4. Seleccionar las columnas categóricas del DataFrame filtrado
df_fact_adicionales = data_geo_filtrado[['dress_code', 'accessibility', 'price', 'franchise']]

# 5. Aplicar OneHotEncoding a las variables categóricas
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
mat_fact_adicionales_ohe = ohe.fit_transform(df_fact_adicionales)

# 6. Verificar las dimensiones antes de la concatenación
print(f"Tamaño de mat_fact_lat_restaurantes: {mat_fact_lat_restaurantes.shape}")
print(f"Tamaño de mat_fact_adicionales_ohe: {mat_fact_adicionales_ohe.shape}")

# 7. Unir la matriz de factores latentes con la matriz de factores adicionales codificados
matriz_hibrida = np.hstack((mat_fact_lat_restaurantes, mat_fact_adicionales_ohe))

# *********** Aquí termina la sección de agregar código *************

# Mostrar la dimensión de la matriz híbrida
print('Dimensión de la matriz híbrida:', matriz_hibrida.shape)





Tamaño de mat_fact_lat_restaurantes: (129, 66)
Tamaño de mat_fact_adicionales_ohe: (129, 11)
Dimensión de la matriz híbrida: (129, 77)


In [18]:
# Recordemos que "restaurante_de_referencia" es "tacos de barbacoa enfrente del Tec".
# Obtener el índice del restaurante
idx = data_geo[data_geo['name'] == restaurante_de_referencia].index[0]

print('Restaurante de referencia:', restaurante_de_referencia)
print('Índice del restaurante de referencia:', idx)

Restaurante de referencia: tacos de barbacoa enfrente del Tec
Índice del restaurante de referencia: 111


## **Ejercicio - 6:**

### **Una vez que tenemos la matriz híbrida, con dicha matriz deberás obtener las similitudes (con respecto a la función coseno) del restaurante de referencia con relación a todos los restaurantes, ordenarlos de mayor a menor con respecto a la similitud del coseno y desplegar los primeros 10 de mayor similitud (sin incluir al restaurante de referencia mismo).**

In [19]:
  # Ejercicio 6

  # ************* Inlcuye aquí tu código:*****************************

# 1. Calcular la matriz de similaridad del coseno
sim_matrix_hibrida = cosine_similarity(matriz_hibrida)

# 2. Definir el restaurante de referencia
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"

# 3. Obtener la lista de nombres de restaurantes en el mismo orden que la matriz híbrida
nombres_restaurantes = UtMx_rating.index.tolist()

# 4. Verificar si el restaurante de referencia existe en la base de datos
if restaurante_de_referencia in nombres_restaurantes:
    # Obtener el índice del restaurante de referencia
    idx_ref = nombres_restaurantes.index(restaurante_de_referencia)

    # Obtener el vector de similaridad del restaurante de referencia con todos los demás
    sim_vector = sim_matrix_hibrida[idx_ref]

    # 5. Crear una lista de tuplas (similitud, nombre_restaurante)
    similitudes = list(zip(sim_vector, nombres_restaurantes))

    # 6. Ordenar de mayor a menor similitud
    similitudes_ordenadas = sorted(similitudes, key=lambda x: x[0], reverse=True)

    # 7. Desplegar los primeros 10 de mayor similitud (excluyendo al restaurante de referencia)
    print("\nTop 10 restaurantes más similares a:", restaurante_de_referencia)
    for sim, nombre in similitudes_ordenadas[1:11]:  # Omitimos el primer resultado porque es el mismo restaurante de referencia
        print(f"Restaurante: {nombre} - Similitud: {sim:.4f}")
else:
    print(f'El restaurante "{restaurante_de_referencia}" no se encuentra en la base de datos.')

# *********** Aquí termina la sección de agregar código *************



Top 10 restaurantes más similares a: tacos de barbacoa enfrente del Tec
Restaurante: tacos abi - Similitud: 0.9683
Restaurante: little pizza Emilio Portes Gil - Similitud: 0.8122
Restaurante: Carnitas Mata  Calle 16 de Septiembre - Similitud: 0.7302
Restaurante: Carreton de Flautas y Migadas - Similitud: 0.7238
Restaurante: vips - Similitud: 0.6772
Restaurante: puesto de gorditas - Similitud: 0.6728
Restaurante: cafe ambar - Similitud: 0.6567
Restaurante: Hamburguesas saul - Similitud: 0.6559
Restaurante: sirloin stockade - Similitud: 0.6365
Restaurante: Taqueria EL amigo  - Similitud: 0.6013


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular comenta las diferencias que encuentras entre un sistema de recomendación basado solamente en la factorización SVD y uno híbrido.**

++++++++ Inicia la sección de agregar texto: +++++++++++


En esta actividad, implementamos un sistema de recomendación utilizando reducción de dimensionalidad con SVD y posteriormente mejoramos su desempeño con un sistema híbrido que incorpora características adicionales de los restaurantes mediante One-Hot Encoding.

Diferencias entre un sistema basado solo en SVD y uno híbrido:

Factorización SVD (Sistema basado en filtrado colaborativo)

Se basa únicamente en la matriz de utilidad (interacciones usuario-restaurante).
Identifica patrones latentes en las calificaciones dadas por los usuarios.
Su desempeño depende en gran medida de la cantidad y calidad de las calificaciones disponibles.
Tiene problemas con el Cold Start, ya que no puede hacer recomendaciones para restaurantes sin calificaciones previas.

Sistema Híbrido (SVD + Factores adicionales)

Combina la información de SVD con atributos adicionales de los restaurantes (dress_code, accessibility, price, franchise).
Puede hacer recomendaciones incluso para restaurantes sin calificaciones previas (mitiga el problema del Cold Start).
Permite que las recomendaciones sean más precisas al considerar aspectos no capturados en las calificaciones, como el tipo de comida o la accesibilidad.
Mejora la capacidad del sistema para encontrar similitudes entre restaurantes con atributos en común.

Conclusión

El sistema híbrido mejora significativamente las recomendaciones en comparación con un sistema basado únicamente en SVD. Al incorporar información adicional sobre los restaurantes, se logran recomendaciones más robustas y explicables, permitiendo una mayor personalización y precisión en la recomendación de restaurantes a los usuarios.


++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**